## 최적의 알고리즘 찾기
- 알고리즘 선정 : 각각의 알고리즘으로 정답률을 구하고 비교
- 알고리즘 평가 : 크로스 밸리데이션

### 각 알고리즘의 정답률

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import all_estimators

iris = load_iris()

y = iris.target
x = iris.data

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [7]:
# 모든 classifier 알고리즘 추출
allAlgorithms = all_estimators(type_filter='classifier')

for name, algorithm in allAlgorithms:
    try:
        clf = algorithm()
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(f'{name}의 정답률 :', accuracy_score(y_test, y_pred))
    except:
        continue

AdaBoostClassifier의 정답률 : 0.9
BaggingClassifier의 정답률 : 0.9
BernoulliNB의 정답률 : 0.26666666666666666
CalibratedClassifierCV의 정답률 : 0.8333333333333334
CategoricalNB의 정답률 : 0.9333333333333333
ComplementNB의 정답률 : 0.6
DecisionTreeClassifier의 정답률 : 0.9
DummyClassifier의 정답률 : 0.26666666666666666
ExtraTreeClassifier의 정답률 : 0.9333333333333333
ExtraTreesClassifier의 정답률 : 0.9
GaussianNB의 정답률 : 0.9
GaussianProcessClassifier의 정답률 : 0.9666666666666667
GradientBoostingClassifier의 정답률 : 0.9333333333333333
HistGradientBoostingClassifier의 정답률 : 0.9
KNeighborsClassifier의 정답률 : 0.9666666666666667
LabelPropagation의 정답률 : 0.9333333333333333
LabelSpreading의 정답률 : 0.9333333333333333
LinearDiscriminantAnalysis의 정답률 : 1.0
LinearSVC의 정답률 : 0.9666666666666667
LogisticRegression의 정답률 : 0.9666666666666667
LogisticRegressionCV의 정답률 : 0.9333333333333333
MLPClassifier의 정답률 : 0.9333333333333333
MultinomialNB의 정답률 : 0.7666666666666667
NearestCentroid의 정답률 : 0.9333333333333333
NuSVC의 정답률 : 0.9666666666666667
PassiveAggress

### 크로스 밸리데이션

In [8]:
from sklearn.model_selection import KFold, cross_val_score

kfold_cv = KFold(n_splits=5, shuffle=True)

for name, algorithm in allAlgorithms:
    try:
        clf = algorithm()
        
        # score 메서드를 가진 객체 대상으로
        if hasattr(clf, 'score'):
            scores = cross_val_score(clf, x, y, cv=kfold_cv)
            print(f'{name}의 정답률 :')
            print(scores)
    except:
        continue

AdaBoostClassifier의 정답률 :
[0.96666667 0.96666667 0.96666667 0.86666667 0.9       ]
BaggingClassifier의 정답률 :
[0.93333333 0.93333333 0.93333333 0.93333333 1.        ]
BernoulliNB의 정답률 :
[0.3        0.23333333 0.23333333 0.3        0.2       ]
CalibratedClassifierCV의 정답률 :
[0.8        0.86666667 0.96666667 0.96666667 0.93333333]
CategoricalNB의 정답률 :
[0.93333333 0.9        0.96666667 0.9        0.93333333]
ComplementNB의 정답률 :
[0.53333333 0.6        0.83333333 0.63333333 0.73333333]
DecisionTreeClassifier의 정답률 :
[0.96666667 0.93333333 0.96666667 0.93333333 1.        ]
DummyClassifier의 정답률 :
[0.2        0.2        0.23333333 0.23333333 0.23333333]
ExtraTreeClassifier의 정답률 :
[1.         0.83333333 0.9        0.9        0.96666667]
ExtraTreesClassifier의 정답률 :
[0.96666667 1.         0.9        0.93333333 0.93333333]
GaussianNB의 정답률 :
[0.96666667 0.96666667 0.93333333 0.96666667 0.93333333]
GaussianProcessClassifier의 정답률 :
[0.96666667 1.         1.         0.93333333 0.93333333]
GradientBoosting

## 최적의 매개변수 찾기

### 그리드 서치

In [11]:
%%time

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = [
    {'C':[1, 10, 100, 1000], 'kernel':['linear', 'rbf', 'sigmoid'], 'gamma':[0.001, 0.0001]}
]

# 그리드 서치
kfold_cv = KFold(n_splits=5, shuffle=True)
clf = GridSearchCV(SVC(), param_grid=parameters, cv=kfold_cv)
clf.fit(x_train, y_train)
print('최적 매개 변수 :', clf.best_params_)

# 평가
y_pred = clf.predict(x_test)
print('최종 정답률 :', accuracy_score(y_test, y_pred))

최적 매개 변수 : {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
최종 정답률 : 1.0
Wall time: 160 ms


### RandomizedSearchCV
- GridSearchCV는 각 파라미터 값을 리스트 형태로 미리 주어준다면
- RandomizedSearchCV는 각 파라미터 값의 범위를 준다

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_dist = {'C':randint(low=1, high=1000), 'kernel':['linear', 'rbf', 'sigmoid'], 'gamma':uniform(0.0001, 0.01)}

kfold_cv = KFold(n_splits=5, shuffle=True)
clf = RandomizedSearchCV(SVC(), param_distributions=param_dist, cv=kfold_cv)
clf.fit(x_train, y_train)
print('최적 매개 변수 :', clf.best_params_)

# 평가
y_pred = clf.predict(x_test)
print('최종 정답률 :', accuracy_score(y_test, y_pred))

최적 매개 변수 : {'C': 313, 'gamma': 0.006029654637724036, 'kernel': 'sigmoid'}
최종 정답률 : 0.9666666666666667
